In [16]:
import pandas as pd
temp_data = pd.read_csv("NYSE_monthly.txt")

# Load timeseries data

In [20]:
temp_data.Month = pd.to_datetime(temp_data.Month)
temp_data.set_index('Month', inplace=True)

In [21]:
temp_data.head()

,monthly_return
Month,
1961-01-01,0.082
1961-02-01,0.059
1961-03-01,0.050
1961-04-01,0.008
1961-05-01,0.042


- Resampling
- Selecting and slicing
- Null values

# Visualizing time series data